In [1]:
import requests, pandas as pd, numpy as np
from requests import session
from bs4 import BeautifulSoup

In [2]:
dfsi=pd.read_csv('dfsi.csv',sep=';')

In [3]:
dfsi2=pd.read_csv('dfsi2.csv',sep=';')

In [4]:
dfsi['nr_alkalmazottak']=dfsi['nr']
dfsi2['nr_arbevetel']=dfsi2['nr']

In [5]:
data=dfsi.set_index('Cégnév').join(dfsi2.set_index('Cégnév'),how='outer',lsuffix='_left', rsuffix='_right')

In [6]:
data['Cím']=data[['cms_right','cms_left']].T.ffill().bfill().T['cms_right']
data['Koord']=data[['coords_right','coords_left']].T.ffill().bfill().T['coords_right']
data['Kw']=data[['kws_right','kws_left']].T.ffill().bfill().T['kws_right']
data['Régió']=data[['region_right','region_left']].T.ffill().bfill().T['region_right']

In [7]:
data=data.drop(['0_left', 'region_left','nr_left', 'coords_left', 'kws_left', 'cms_left',
       '0_right', 'region_right', 'nr_right', 'coords_right', 'kws_right', 'cms_right',
        'Unnamed: 0_left','Unnamed: 0_right'],axis=1)

In [8]:
lat=[]
lon=[]
for i in range(len(data.index)):
    k=data.loc[data.index[i]]['Koord'].replace("'",'').replace('[','')\
    .replace(']','').replace(' ','').split(',')
    lon.append(k[0])
    lat.append(k[1])

In [9]:
data['Latitude']=lat
data['Longitude']=lon

In [10]:
data=data.drop('Koord',axis=1)

In [11]:
data

,Alkalmazottak száma 2017,Alkalmazottak száma 2016,Alkalmazottak száma 2015,Alkalmazottak száma 2014,nr_alkalmazottak,Árbevétel 2017 (RON),Árbevétel 2016 (RON),Árbevétel 2015 (RON),Árbevétel 2014 (RON),nr_arbevetel,Cím,Kw,Régió,Latitude,Longitude
Cégnév,,,,,,,,,,,,,,,
A M C SRL,NaN,NaN,NaN,NaN,NaN,27 881 270,24 390 005,22 173 039,18 131 027,20.0,"527100, H\xeddv\xe9g, Rom\xe1n u., 203\t","['nagykereskedelem', '', '""']",also-haromszek,25.5892160,45.8391350
ABC IMPEX SRL,145,132,132,120,20.0,36 539 530,33 071 733,31 956 498,30 363 973,20.0,"535600, Sz\xe9kelyudvarhely, R\xe1k\xf3czi Fer...","['csomagoloanyagok', 'nyomdak', '', '""']",udvarhelyszek,25.290034,46.289768
ABRAZIV SRL,NaN,NaN,NaN,NaN,NaN,5 483 275,0,0,0,49.0,"535500, Gyergy\xf3szentmikl\xf3s, \xc1llom\xe1...","['szerszamgepek', '', '""']",gyergyoszek,25.5751650,46.7174250
ADIMAG COM IMPEX SRL,NaN,NaN,NaN,NaN,NaN,57 106 224,48 827 471,47 437 389,44 031 925,36.0,"540190, Marosv\xe1s\xe1rhely, Szabads\xe1g u.,...","['belsoepiteszeti-anyagok', 'epitoanyagok', 'f...",marosszek,24.548819,46.537905
AFEROM TRANS SRL,NaN,NaN,NaN,NaN,NaN,13 341 348,13 592 056,12 531 454,12 096 492,30.0,"530153, Cs\xedkszereda, Ak\xe1cfa u., 3A\t","['belfoldi-teherfuvarozas', 'kulfoldi-teherfuv...",csikszek,25.786071,46.361412
AGER SRL,NaN,NaN,NaN,NaN,NaN,14 549 472,10 629 185,10 511 810,10 322 534,45.0,"525300, Bodzafordul\xf3, Kemping u., 29\t","['muanyag-feldolgozas', '', '""']",also-haromszek,26.0291420,45.6671960
AGM ECO CORPORATE SRL,68,36,2,6,50.0,NaN,NaN,NaN,NaN,NaN,"535600, Sz\xe9kelyudvarhely, Cip\xe9szek u., 15\t","['pekaru', 'pekaru-es-peksutemenyek', 'pekseg-...",udvarhelyszek,25.3117290,46.3160230
AGRICO M SRL,NaN,NaN,NaN,NaN,NaN,9 400 301,7 643 351,6 376 915,5 701 521,30.0,"525400, K\xe9zdiv\xe1s\xe1rhely, Ady Endre u.,...","['burgonya-vetomag', 'b\\xfazavetomag', 'etkez...",felso-haromszek,26.1326700,46.0066320
AGRO PAN STAR SRL,NaN,NaN,NaN,NaN,NaN,19 033 775,20 905 425,13 372 911,2 845 996,34.0,"520019, Sepsiszentgy\xf6rgy, Dacz\xf3 u., 6\t","['kenyer', 'kenyer-es-pekaru', 'peksegek', 'pe...",also-haromszek,25.7932580,45.8700010


In [42]:
manual=data[data['Longitude']=='']
data=data[data['Longitude']!='']

In [123]:
data.to_excel('data.xlsx')
manual.to_excel('manual.xlsx')

In [111]:
sectors=pd.read_excel('sectors.xlsx')
sectormap={}
repl={'á':'a','é':'e','í':'i','ó':'o','ú':'u','ü':'u','ű':'u','ő':'o','ö':'o'}
for s in sectors.columns:
    for k in sectors[s].values:
        for j in str(k).replace(' ','').split(','):
            sectormap[j]=s
            for c in repl:
                j=j.replace(c,repl[c])
            sectormap[j]=s

In [120]:
valid=[]
kws=[]
cont=True
ki=-1
for i in range(len(data.index)):
    ks=data.loc[data.index[i]]['Kw'].replace("'",'').replace('[','')\
                .replace(']','').replace(' ','').lower().split(',')[:-2]
    for k in ks:
        if k in sectormap:
            kws.append(sectormap[k])
            break
        k=k.replace('-','')
        if k in sectormap:
            kws.append(sectormap[k])
            break
    if len(kws)<i-1: 
        print('ERROR',ks,i)
        break
    else:
        #print('SUCCES',kws[-1],ks)
        valid.append([data.index[i],kws[-1],ks])

In [121]:
pd.DataFrame(valid).to_excel('valid.xlsx')

Manual overwrite